In [ ]:
from api_request import api_request
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import pandas as pd
import plotly.express as px
from io import StringIO

In [ ]:
# read indicator list
cl_ind = pd.read_csv("./indicator_list/CL_INDICATORS_TRANSMONEE.csv")
# read ecaro country list
ecaro_geo = pd.read_csv("./indicator_list/CL_COUNTRY.csv")

In [ ]:
# dropdown indicators
dd_ind = dcc.Dropdown(
    id="my_ind",
    options=[
        {"label": value, "value": key}
        for key, value in zip(cl_ind.Id,cl_ind.Name)
    ],
    value='NT_ANT_WHZ_PO2'
)
# dropdown countries
dd_geo = dcc.Dropdown(
    id="my_geo",
    options=[
        {"label": value, "value": key}
        for key, value in zip(ecaro_geo.Id,ecaro_geo.Name)
    ],
    value='ALB'
)

In [ ]:
# detect proxy configuration for JupyterHub or Binder
JupyterDash.infer_jupyter_proxy_config()

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
# Build App
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
# Create server variable with Flask server object for use with gunicorn
server = app.server

In [ ]:
# App Layout
app.layout = html.Div([
    html.H2("Check TM DB"),
    html.H6("Browse by Indicator: get latest year reported"),
    html.Div(["Select Indicator", dd_ind]),
    html.Br(),
    html.Div(id="my-output"),
    html.Br(),
    html.H6("Browse by Country: visualize (totals) time series"),
    html.Div(["Select Country", dd_geo]),
    dcc.Graph(id='time-series'),
    # Hidden div inside to share data across
    html.Div(id='store-codes', style={'display': 'none'})
], style={"height":"100vh"})

In [ ]:
# parameters: API-SDMX Helix (latest observation)
api_params = {"lastNObservations":1}
# parameters: API-SDMX Helix (time series)
years = [1950, 2050]
api_params_time = {"startPeriod": str(years[0]), "endPeriod": str(years[1])}
# API headers
api_headers = {
    "Accept": "application/vnd.sdmx.data+csv;version=1.0.0",
    "Accept-Encoding": "gzip",
}
# UNICEF data warehouse TM url endpoint
wh_url = "https://sdmx.data.unicef.org/ws/public/sdmxapi/rest/data/TRANSMONEE"
# TRANSMONEE DB DIMENSIONS
tm_db_dim = ["SEX", "AGE", "RESIDENCE", "WEALTH_QUINTILE"]

In [ ]:
# Define callback: latest year and disagg available
@app.callback(
    Output("my-output", "children"),
    Output("store-codes", "children"),
    Input("my_ind", "value"),
)
def query_db(indicator):
    if indicator:
        query = f".{indicator}...."
        ind_req = api_request(f"{wh_url}/{query}", api_params, api_headers)
        data = pd.read_csv(StringIO(ind_req.text))
        # rename column names: only code
        cols = data.columns.values
        ren_dict = {k: v.split(":")[0] for k, v in zip(cols, cols)}
        data.rename(columns=ren_dict, inplace=True)
        # latest observation
        maxdata = data.TIME_PERIOD.max()
        # available disagg
        disagg_avlbl = [
            dim for dim in tm_db_dim if len(data[dim].unique()) > 1
        ]
        # build message
        text_msg = (
            f"Latest Year: {maxdata}" if not disagg_avlbl else 
            "Latest Year: {}, disaggregation: {}".format(maxdata, ", ".join(disagg_avlbl))
        )
        # data to share
        data_to_share = data.to_json(orient='split')
    else:
        text_msg = "Please select indicator"
        data_to_share = None
        disagg_avlbl = None
    return text_msg, (data_to_share, disagg_avlbl)

In [ ]:
# Define callback to update graph
@app.callback(
    Output("time-series", "figure"),
    Input('store-codes', 'children'),
    Input("my_ind", "value"),
    Input("my_geo", "value"),
)
def query_2_plot(last_obs_data, indicator, country):
    if (not indicator) & (not country):
        return {}
    else:
        # check disagg from stored data
        disagg_avlbl = last_obs_data[1]
        if disagg_avlbl:
            stored_data = pd.read_json(last_obs_data[0], orient='split')
            code_total = {
                k:(
                    "_T" if "_T: Total" in stored_data[k].values else
                    stored_data[k].unique()[0].split(":")[0]
                )
                for k in disagg_avlbl
            }
            dim_val = {
                k:("" if k not in disagg_avlbl else code_total[k]) for k in tm_db_dim
            }
            query = "{}.{}.{}.{}.{}.{}".format(
                country,
                indicator,
                *dim_val.values(),
            )
        else:
            query = f"{country}.{indicator}...."
        ind_req_time = api_request(f"{wh_url}/{query}", api_params_time, api_headers)
        
    if ind_req_time.status_code == 200:
        data = pd.read_csv(StringIO(ind_req_time.text))
        # rename column names: only code
        cols = data.columns.values
        ren_dict = {k: v.split(":")[0] for k, v in zip(cols, cols)}
        data.rename(columns=ren_dict, inplace=True)
        return px.line(
            data,
            x="TIME_PERIOD",
            y="OBS_VALUE",
            hover_name="DATA_SOURCE",
            line_shape="spline",
        ).update_traces(mode="lines+markers")
    else:
        return {}

In [ ]:
# Run app and display result inline in the notebook
app.run_server(mode='inline')